Data 불러와서 나누기

In [2]:
import pickle
with open('./data/train_token.pickle', 'rb') as fs:
    train_token = pickle.load(fs)
with open('./data/dev_token.pickle', 'rb') as fs:
    dev_token = pickle.load(fs)
with open('./data/train_pos.pickle', 'rb') as fs:
    train_pos = pickle.load(fs)
with open('./data/dev_pos.pickle', 'rb') as fs:
    dev_pos = pickle.load(fs)

pos_vocab 만들기

In [78]:
pos_vocab = ['NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP', 'VV', 'VA', 'VX', 'VCP', 'VCN', 'MM', 'MAG', 'MAJ', 'IC', 'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC', 'EP', 'EF', 'EC', 'ETN', 'ETM', 'XPN', 'XSN', 'XSV', 'XSA', 'XR', 'SF', 'SE', 'SSO', 'SSC', 'SC', 'SY'	, 'SL', 'SH', 'SN', 'OTHER', 'UNKNOWN', '<SP>']

In [55]:
# pos_vocab = ['Noun', 'Verb', 'Adjective', 'Determiner', 'Adverb', 'Conjunction', 'Exclamation', 'Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation', 'Foreign', 'Alpha', 'Number', 'Unknown', 'KoreanParticle', 'Hashtag', 'ScreenName', 'Email', 'URL']

# train, dev 데이터에서 품사 집합 구해서 pos_vocab에 추가
# pos_set_train = set([y for x in train_pos for y in x])
# pos_set_dev = set([y for x in dev_pos for y in x])
# pos_set = list(pos_set_train | pos_set_dev)
# vocab = sorted(list(set(vocab) | set(pos_set)))

One_hot 벡터 만들기

In [79]:
def posOnehot(sentence, vocab):
    import numpy as np

    onehot_list = []
    for word in sentence:
        onehot = np.zeros(len(vocab))

        if word in vocab:
            ind = vocab.index(word)
            onehot[ind] += 1 
        elif '+' in word:
            onehot[vocab.index('OTHER')] += 1
        else:
            onehot[vocab.index('UNKNOWN')] += 1
            
        onehot_list.append(onehot)
    return onehot_list

train_onehot = [posOnehot(pos, pos_vocab) for pos in train_pos]
dev_onehot = [posOnehot(pos, pos_vocab) for pos in dev_pos]

In [143]:
def makePosDict(token, key_list, onehot_vector):
    output = {}
    for i, sentence in enumerate(token):
        for j, word in enumerate(sentence):
            key = key_list[key_list.index(word)]
            output[key] = onehot_vector[i][j]
    return output

key_list_train = list(set([y for x in train_token for y in x]))
key_list_dev = list(set([y for x in dev_token for y in x]))

train_dict = makePosDict(train_token, key_list_train, train_onehot)
dev_dict = makePosDict(dev_token, key_list_dev, dev_onehot) 

In [145]:
with open('train_dict_sp.pickle', 'wb') as fs:
    pickle.dump(train_dict, fs)
with open('dev_dict_sp.pickle', 'wb') as fs:
    pickle.dump(dev_dict, fs)